In [36]:
!pip install pandas dash

In [37]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [38]:
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [39]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
launch_sites = spacex_df.groupby(['Launch Site'], as_index = False).mean()
launch_sites_dict = [{'label': 'All Sites', 'value': 'ALL'}]

for i in range(len(launch_sites)):

    launch_site_name = launch_sites.loc[i, 'Launch Site']
    launch_sites_dict.append({'label': launch_site_name, 'value': launch_site_name})

In [40]:
# Create a dash application
app = dash.Dash(__name__)

In [41]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                html.Div(dcc.Dropdown(id='site-dropdown',options=launch_sites_dict,value='ALL',placeholder='Select a Launch Site here',searchable=True)),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                html.Div(dcc.RangeSlider(id='payload-slider',min=0,max=10000,step=1000,value=[min_payload, max_payload])),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [42]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output("success-pie-chart", "figure"), 
    [Input("site-dropdown", "value")])
def generate_chart(value):
    
    if value == 'ALL':
        spacex_df_sum = spacex_df.groupby('Launch Site', as_index = False).sum()
        fig = px.pie(spacex_df_sum, title='Total Success Launches By Site', values='class', names='Launch Site')
    else:
        spacex_df_count = pd.DataFrame(spacex_df)
        for i in range(len(spacex_df_count)):
            
            if spacex_df_count.loc[i, 'Launch Site'] != value:
                spacex_df_count.drop(i, inplace=True)
        spacex_df_count = spacex_df_count.groupby('class', as_index = False).count()
        fig = px.pie(spacex_df_count, title='Total Success Launches for site '+value, values='Launch Site', names='class')
    
    return fig

In [43]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output("success-payload-scatter-chart", "figure"), 
    [Input("site-dropdown", "value"),Input("payload-slider", "value")]
    )
def generate_scatter_chart(site, payload):
    low, high = payload
    mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    if site == 'ALL':
        fig = px.scatter(spacex_df[mask], x='Payload Mass (kg)', y='class', color='Booster Version Category')
    else:
        spacex_df_filtered = pd.DataFrame(spacex_df)
        mask = (spacex_df_filtered['Payload Mass (kg)'] > low) & (spacex_df_filtered['Payload Mass (kg)'] < high)
        for i in range(len(spacex_df_filtered)):
            
            if spacex_df_filtered.loc[i, 'Launch Site'] != site:
                spacex_df_filtered.drop(i, inplace=True)
        fig = px.scatter(spacex_df_filtered[mask], x='Payload Mass (kg)', y='class', color='Booster Version Category')
    return fig

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
